In [ ]:
!pip install -q selenium
!apt-get -qq update
!apt install -qq chromium-chromedriver sqlite3
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [2]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
urls = {
  "primary": "https://overframe.gg/tier-list/primary-weapons/",
  "secondary": "https://overframe.gg/tier-list/secondary-weapons/",
  "melee": "https://overframe.gg/tier-list/melee-weapons/"
} 

In [4]:
from selenium.webdriver.common.by import By
import pandas as pd

def getTier(url, wType):
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  wd.get(url)

  tierContainers = wd.find_elements(by=By.CLASS_NAME, value="TierList_tierContainer__psIt4")
  wStorage = []
  tStorage = []

  for tierContainer in tierContainers:
    currTier = tierContainer.find_element(By.CLASS_NAME, "TierList_tierTitle__PdTC9").text
    weapons = tierContainer.find_elements(By.TAG_NAME, "figcaption")
    for weapon in weapons:
      wStorage.append(weapon.text)
      tStorage.append(currTier)

  d = {
      "type": wType,
      "weapon": wStorage,
      "tier": tStorage,
  }
  df = pd.DataFrame(data=d)

  return df

### Storing data

In [5]:
priFrame = getTier(urls["primary"], "primary")
secFrame = getTier(urls["secondary"], "secondary")
melFrame = getTier(urls["melee"], "melee")

In [6]:
frame = priFrame.append(secFrame, ignore_index=True)
frame = frame.append(melFrame, ignore_index=True)

In [7]:
frame

,type,weapon,tier
0,primary,KUVA ZARR,S Tier - Prime Time
1,primary,NATARUK,S Tier - Prime Time
2,primary,PROBOSCIS CERNOS,S Tier - Prime Time
3,primary,KUVA BRAMMA,S Tier - Prime Time
4,primary,CEDO,S Tier - Prime Time
...,...,...,...
478,melee,MK1-FURAX,D Tier - Weak Picks
479,melee,KESTREL,D Tier - Weak Picks
480,melee,FURAX,D Tier - Weak Picks
481,melee,KOGAKE,D Tier - Weak Picks


### SQL Part

In [8]:
import sqlite3
 
conn = sqlite3.connect('test.db')

In [9]:
frame.to_sql('weapons', con=conn)

In [10]:
!sqlite3 test.db .dump >> database.sql
!tail -n 10 database.sql

INSERT INTO weapons VALUES(475,'melee','MACHETE','D Tier - Weak Picks');
INSERT INTO weapons VALUES(476,'melee','FANG','D Tier - Weak Picks');
INSERT INTO weapons VALUES(477,'melee','BO','D Tier - Weak Picks');
INSERT INTO weapons VALUES(478,'melee','MK1-FURAX','D Tier - Weak Picks');
INSERT INTO weapons VALUES(479,'melee','KESTREL','D Tier - Weak Picks');
INSERT INTO weapons VALUES(480,'melee','FURAX','D Tier - Weak Picks');
INSERT INTO weapons VALUES(481,'melee','KOGAKE','D Tier - Weak Picks');
INSERT INTO weapons VALUES(482,'melee','ANKYROS','D Tier - Weak Picks');
CREATE INDEX "ix_weapons_index"ON "weapons" ("index");
COMMIT;
